# BTS On-Time Flights — EDA, Cleaning, and Feature Engineering

This notebook prepares 2024 BTS on-time performance data for downstream BI by:
- Standardizing column names and dtypes
- Cleaning nulls/sparse columns and applying schema
- Engineering features (FE1–FE6) per instructions
- Exporting a clean, sampled Parquet for analysis

## Setup & Imports
Imports used throughout; keep code readable and efficient.

In [ ]:
import os, math, re, sys, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 160)

# Light style for quick checks
sns.set_context('notebook')

# Utility: show small previews consistently
def preview(df, n=3):
    try:
        display(df.head(n))
    except NameError:
        return df.head(n)

preview(pd.DataFrame({'ok': [1,2,3]}))

## Globals & Configuration
Collect all tweakable variables in one place for easy modification.

In [ ]:
# Paths
DATA_DIR = Path('data')
FLIGHTS_DIR = DATA_DIR / 'flights'
DIMS_DIR = DATA_DIR / 'dims'
OUTPUT_DIR = DATA_DIR / 'output'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Cleaning configuration
SPARSE_NULL_THRESHOLD = 0.80  # drop columns with >= 80% nulls
CRITICAL_FIELDS = ['depdelayminutes', 'depdel15']  # rows missing these are dropped

# Sampling configuration (FE6)
SAMPLE_RATE = 0.10  # 10% stratified sample
RANDOM_STATE = 42
SAMPLE_KEYS = ['month', 'iata_code_reporting_airline', 'depdel15']

# Output files
OUTPUT_PARQUET = OUTPUT_DIR / 'flights_2024_clean_sampled.parquet'

# Plotting toggles (set False to skip generating figures)
SHOW_PLOTS = True

# Test mode for fast headless runs (CLI/CI)
TEST_MODE = os.environ.get('TEST_MODE', '0') == '1'
READ_NROWS = 50000 if TEST_MODE else None
if TEST_MODE:
    SHOW_PLOTS = False
print('Config — TEST_MODE:', TEST_MODE, '| READ_NROWS:', READ_NROWS)

# Helper: column standardization (lowercase; replace spaces with underscores)
def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
    return df

# Quick check of directories
print('Flights path exists:', FLIGHTS_DIR.exists())
print('Dims path exists:', DIMS_DIR.exists())
preview(pd.DataFrame({'flights_dir': [str(FLIGHTS_DIR)], 'dims_dir': [str(DIMS_DIR)]}))

## Discover Input Files
List available monthly CSVs and dims files.

In [ ]:
flight_files = sorted(FLIGHTS_DIR.glob('*.csv'))
dims_airport_path = DIMS_DIR / 'T_MASTER_CORD.csv'
dims_airline_path = DIMS_DIR / 'L_AIRLINE_ID.csv'

print(f'Total flight CSVs: {len(flight_files)}')
preview(pd.DataFrame({'file': [p.name for p in flight_files]}).head(3))
preview(pd.DataFrame({'dims_airport': [dims_airport_path.name], 'dims_airline': [dims_airline_path.name]}))

## Load Flights (All Months)
Load monthly CSVs, standardize column names, and concatenate. For memory safety, we defer heavy casting until after pruning.

In [ ]:
def read_flight_csv(path: Path) -> pd.DataFrame:
    # Read as strings first to avoid mixed dtypes; cast later.
    df = pd.read_csv(path, dtype=str, low_memory=False, nrows=READ_NROWS)
    df = standardize_columns(df)
    return df

print('Reading', len(flight_files), 'files with nrows =', READ_NROWS)
flights_list = [read_flight_csv(p) for p in flight_files]
df = pd.concat(flights_list, ignore_index=True) if flights_list else pd.DataFrame()
print('Rows:', len(df), 'Cols:', len(df.columns))
preview(df)

## Quick EDA: Missingness & Basic Sanity
Understand nulls to inform pruning and dtype casting.

In [ ]:
if not df.empty:
    missing_frac = df.isna().mean().sort_values(ascending=False)
    preview(missing_frac.to_frame('missing_frac'))
    preview(df[['flightdate','month','origin','dest','crsdeptime','deptime','depdelay','depdelayminutes','depdel15']].head(3))
else:
    preview(pd.DataFrame({'note': ['no data loaded']}))

## DE1: Drop Rows Missing Critical Fields
Remove rows where `depdelayminutes` or `depdel15` is null/blank.

In [ ]:
def drop_critical_nulls(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in CRITICAL_FIELDS:
        if c in df.columns:
            df[c] = df[c].replace('', np.nan)
            df = df[df[c].notna()]
    return df

before = len(df)
df = drop_critical_nulls(df)
after = len(df)
print('Dropped rows (critical nulls):', before - after, 'Remaining:', after)
preview(df.head(3))

## DE2: Drop Sparse Columns (>= 80% Nulls)
Prune legacy/unused fields. Threshold configurable via `SPARSE_NULL_THRESHOLD`.

In [ ]:
def drop_sparse_columns(df: pd.DataFrame, threshold: float) -> pd.DataFrame:
    frac = df.isna().mean()
    to_drop = frac[frac >= threshold].index.tolist()
    print(f'Dropping {len(to_drop)} sparse columns (>= {threshold:.0%} nulls)')
    return df.drop(columns=to_drop)

df = drop_sparse_columns(df, SPARSE_NULL_THRESHOLD)
preview(pd.Series(df.columns[:12]))

## DE3: Dtype Casting per Dictionary/Best Guess
Set canonical types for key fields; parse dates and numeric fields.

In [ ]:
def to_int(series):
    return pd.to_numeric(series, errors='coerce').astype('Int64')

def to_float(series):
    return pd.to_numeric(series, errors='coerce').astype(float)

# Parse date
if 'flightdate' in df.columns:
    df['flightdate'] = pd.to_datetime(df['flightdate'], errors='coerce')

# Numeric casts for key fields
for c in ['year','quarter','month','dayofmonth','dayofweek','originairportid','destairportid','dot_id_reporting_airline']:
    if c in df.columns:
        df[c] = to_int(df[c])

for c in ['depdelay','depdelayminutes','arrdelay','arrdelayminutes','taxiout','taxiin','airtime','distance']:
    if c in df.columns:
        df[c] = to_float(df[c])

# Keep string/object for codes
for c in ['origin','dest','iata_code_reporting_airline','reporting_airline','tail_number']:
    if c in df.columns:
        df[c] = df[c].astype('string')

preview(df[['flightdate','month','origin','dest','depdelayminutes','depdel15']].head(3))

## FE1: Day-Part (Scheduled and Actual) with Midnight Rollover
Create `daypart_sched` from `crsdeptime` and `daypart_actual` from `deptime` with rollover handling.

In [ ]:
from datetime import datetime, date, time, timedelta

def parse_hhmm(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == '':
        return np.nan
    # Handle decimal-like values (e.g., '856.00')
    s = s.split('.')[0]
    s = s.zfill(4) if s.isdigit() else re.sub(r'[^0-9]', '', s).zfill(4)
    hh = int(s[:2])
    mm = int(s[2:])
    if hh == 24 and mm == 0:
        hh, mm = 0, 0
    hh = max(0, min(23, hh))
    mm = max(0, min(59, mm))
    return time(hh, mm)

def daypart_from_time(t: time):
    if pd.isna(t):
        return np.nan
    h = t.hour
    if 0 <= h <= 3: return 'night'
    if 4 <= h <= 7: return 'early_morning'
    if 8 <= h <= 11: return 'morning'
    if 12 <= h <= 15: return 'afternoon'
    if 16 <= h <= 19: return 'late_afternoon'
    return 'evening'

def compute_dayparts(df: pd.DataFrame) -> pd.DataFrame:
    # Reset index to ensure positional access works reliably after drops
    df = df.copy().reset_index(drop=True)
    crs_t = df['crsdeptime'].apply(parse_hhmm) if 'crsdeptime' in df else pd.Series([np.nan] * len(df))
    dep_t = df['deptime'].apply(parse_hhmm) if 'deptime' in df else pd.Series([np.nan] * len(df))

    # Scheduled daypart
    df['daypart_sched'] = [daypart_from_time(t) for t in crs_t]

    # Actual daypart with midnight rollover if depdelay >= 0 and deptime < crsdeptime
    if 'flightdate' in df.columns:
        sched_dt = [pd.NaT] * len(df)
        act_dt = [pd.NaT] * len(df)
        for i in range(len(df)):
            # Use positional access to avoid KeyError with non-consecutive indexes
            fd = df.iloc[i]['flightdate']
            ct = crs_t.iloc[i] if isinstance(crs_t, pd.Series) else crs_t
            at = dep_t.iloc[i] if isinstance(dep_t, pd.Series) else dep_t
            if not pd.isna(fd) and isinstance(ct, time):
                sched_dt[i] = datetime.combine(fd.date() if isinstance(fd, pd.Timestamp) else fd, ct)
            if not pd.isna(fd) and isinstance(at, time):
                base = datetime.combine(fd.date() if isinstance(fd, pd.Timestamp) else fd, at)
                # Rollover rule
                dep_delay = df.iloc[i]['depdelay'] if 'depdelay' in df.columns else np.nan
                if not pd.isna(dep_delay) and dep_delay >= 0 and isinstance(ct, time) and at < ct:
                    base = base + timedelta(days=1)
                act_dt[i] = base
        df['daypart_actual'] = [daypart_from_time(d.time()) if not pd.isna(d) else np.nan for d in act_dt]
    else:
        df['daypart_actual'] = np.nan
    return df

df = compute_dayparts(df)
preview(df[['flightdate','crsdeptime','deptime','depdelay','daypart_sched','daypart_actual']].head(3))

## FE2: Origin Airport Tiers (Top 20%, Next 50%, Bottom 30%)
Group by `origin` across all flights (including cancelled); assign tiers by airport percentiles.

In [ ]:
def assign_origin_tiers(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    counts = df.groupby('origin', dropna=False).size().reset_index(name='flights')
    counts = counts.sort_values(['flights','origin'], ascending=[False, True]).reset_index(drop=True)
    counts['airport_rank'] = np.arange(1, len(counts)+1)
    counts['airport_pct'] = counts['airport_rank'] / len(counts)
    def tier(p):
        if p <= 0.20: return 'Tier 1'
        if p <= 0.70: return 'Tier 2'
        return 'Tier 3'
    counts['origin_tier'] = counts['airport_pct'].apply(tier)
    return df.merge(counts[['origin','origin_tier']], on='origin', how='left')

df = assign_origin_tiers(df)
preview(df[['origin','origin_tier']].head(3))

## FE3: Late/On-Time Flags
Create `is_late_departure` from `depdel15` and `is_on_time_departure = 1 - is_late_departure`.

In [ ]:
def add_departure_flags(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['depdel15'] = pd.to_numeric(df['depdel15'], errors='coerce')
    df['is_late_departure'] = (df['depdel15'] == 1).astype('Int8')
    df['is_on_time_departure'] = (1 - df['is_late_departure'].fillna(0)).astype('Int8')
    return df

df = add_departure_flags(df)
preview(df[['depdel15','depdelayminutes','is_late_departure','is_on_time_departure']].head(3))

## FE4: Airport Coordinates (Origin/Dest)
Join `T_MASTER_CORD.csv` on airport IDs, filter `airport_is_latest == 1`, and keep only lat/long.

In [ ]:
airports = pd.read_csv(dims_airport_path, dtype=str)
airports = standardize_columns(airports)
airports = airports[airports['airport_is_latest'] == '1']
airports = airports[['airport_id','latitude','longitude']].copy()
airports['airport_id'] = pd.to_numeric(airports['airport_id'], errors='coerce').astype('Int64')
airports['latitude'] = pd.to_numeric(airports['latitude'], errors='coerce')
airports['longitude'] = pd.to_numeric(airports['longitude'], errors='coerce')

df = df.merge(airports.rename(columns={'airport_id':'originairportid', 'latitude':'origin_lat', 'longitude':'origin_long'}),
               on='originairportid', how='left')
df = df.merge(airports.rename(columns={'airport_id':'destairportid', 'latitude':'dest_lat', 'longitude':'dest_long'}),
               on='destairportid', how='left')
preview(df[['origin','origin_lat','origin_long','dest','dest_lat','dest_long']].head(3))

## FE5: Airline Name (Join on IATA Code)
Parse IATA code and airline name from `L_AIRLINE_ID.csv` descriptions, then join to flights.

In [ ]:
airlines = pd.read_csv(dims_airline_path, dtype=str)
airlines = standardize_columns(airlines)
# Expect columns: 'code' (numeric id), 'description' like 'Southwest Airlines Co.: WN'
def split_airline_desc(s):
    if pd.isna(s):
        return pd.Series({'airline_name': np.nan, 'iata_code': np.nan})
    parts = str(s).split(':')
    if len(parts) >= 2:
        name = parts[0].strip()
        code = parts[1].strip()
        return pd.Series({'airline_name': name, 'iata_code': code})
    return pd.Series({'airline_name': str(s).strip(), 'iata_code': np.nan})

airlines[['airline_name','iata_code']] = airlines['description'].apply(split_airline_desc)
airlines['iata_code'] = airlines['iata_code'].str.upper()
airline_lookup = airlines[['iata_code','airline_name']].dropna(subset=['iata_code']).drop_duplicates()

df = df.merge(airline_lookup, left_on='iata_code_reporting_airline', right_on='iata_code', how='left')
df = df.drop(columns=['iata_code'])
preview(df[['iata_code_reporting_airline','airline_name']].head(3))

## FE6: Stratified Down-Sampling (10%)
Stratify by `month`, `iata_code_reporting_airline`, and `depdel15` to preserve seasonality, carrier mix, and delay balance.

In [ ]:
def stratified_sample(df: pd.DataFrame, keys, frac: float, random_state: int) -> pd.DataFrame:
    rng = np.random.default_rng(random_state)
    def take_sample(g):
        n = len(g)
        k = max(1, int(math.ceil(n * frac)))
        return g.sample(n=min(k, n), random_state=random_state, replace=False)
    return df.groupby(keys, dropna=False, as_index=False, group_keys=False).apply(take_sample)

df_sampled = stratified_sample(df, SAMPLE_KEYS, SAMPLE_RATE, RANDOM_STATE) if not df.empty else df
print('Sampled rows:', len(df_sampled), 'out of', len(df))
preview(df_sampled[['month','iata_code_reporting_airline','depdel15']].head(3))

## DE6: Final Cleanup
Drop redundant and ambiguous columns (e.g., raw flags, IDs used only for joins), keeping BI-useful fields.

In [ ]:
UNWANTED_COLS = [
    'tail_number', 'depdel15', 'reporting_airline', 'dot_id_reporting_airline', 'iata_code_reporting_airline'
]  # extend as needed

def drop_unwanted(df: pd.DataFrame, cols) -> pd.DataFrame:
    existing = [c for c in cols if c in df.columns]
    print('Dropping unwanted cols:', existing)
    return df.drop(columns=existing)

df_final = drop_unwanted(df_sampled, UNWANTED_COLS)
preview(pd.Series(sorted(df_final.columns)).head(12))

## Export to Parquet
Write the engineered, sampled dataset to `data/output`.

In [ ]:
df_final.to_parquet(OUTPUT_PARQUET, index=False)
preview(pd.DataFrame({'written_to': [str(OUTPUT_PARQUET)]}))

## Validation Visuals
Quick plots to sanity-check distributions and feature derivations.

In [ ]:
if SHOW_PLOTS and not df.empty:
    fig, axes = plt.subplots(1, 3, figsize=(15,4))
    sns.countplot(data=df, x='daypart_sched', ax=axes[0], order=['night','early_morning','morning','afternoon','late_afternoon','evening'])
    axes[0].set_title('Scheduled Dayparts')
    sns.countplot(data=df, x='is_late_departure', ax=axes[1])
    axes[1].set_title('Late Departure Flag')
    sns.countplot(data=df, x='origin_tier', ax=axes[2], order=['Tier 1','Tier 2','Tier 3'])
    axes[2].set_title('Origin Tiers')
    plt.tight_layout()
    plt.show()
    preview(df[['daypart_sched','is_late_departure','origin_tier']].head(3))
else:
    preview(pd.DataFrame({'note': ['plots skipped or no data']}))